In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [3]:
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "bigdata_airbnb",
    "user": "postgres",
    "password": "1234"
}

engine = create_engine(
    f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
    f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)


In [9]:
query = """
SELECT *
FROM raw.airbnb_combined_raw
"""

df = pd.read_sql(query, engine)
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne,None,28804,35.65146,-82.62792,Private room,60.0,1.0,138.0,None,1.14,1.0,0.0
1,80905,French Chic Loft,427027,Celeste,None,28801,35.59779,-82.55540,Entire home/apt,470.0,1.0,114.0,None,1.03,11.0,288.0
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,Lisa,None,28801,35.60670,-82.55563,Entire home/apt,75.0,30.0,89.0,None,0.81,2.0,298.0
3,155305,Cottage! BonPaul + Sharky's Hostel,746673,BonPaul,None,28806,35.57864,-82.59578,Entire home/apt,90.0,1.0,267.0,None,2.39,5.0,0.0
4,160594,Historic Grove Park,769252,Elizabeth,None,28801,35.61442,-82.54127,Private room,125.0,30.0,58.0,None,0.52,1.0,0.0


In [10]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328629 entries, 0 to 328628
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              328629 non-null  int64  
 1   name                            328351 non-null  object 
 2   host_id                         328629 non-null  int64  
 3   host_name                       328190 non-null  object 
 4   neighbourhood_group             212755 non-null  object 
 5   neighbourhood                   328613 non-null  object 
 6   latitude                        328621 non-null  float64
 7   longitude                       328621 non-null  float64
 8   room_type                       328629 non-null  object 
 9   price                           226030 non-null  float64
 10  minimum_nights                  328207 non-null  float64
 11  number_of_reviews               328446 non-null  float64
 12  last_review     

In [11]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,3.286290e+05,3.286290e+05,328621.000000,328621.000000,226030.000000,3.282070e+05,328446.000000,264148.000000,328310.000000,327749.000000
mean,2.661894e+07,1.544161e+10,37.244136,-94.082649,219.716529,3.142012e+02,32.316685,1.412597,13.968907,153.865278
std,1.431297e+07,2.780729e+10,6.146765,25.630366,570.353609,1.745525e+05,59.655709,1.704498,46.210328,138.946548
min,1.090000e+02,2.300000e+01,18.920990,-159.714900,0.000000,1.000000e+00,0.000000,0.010000,1.000000,0.000000
25%,1.513319e+07,2.767398e+07,34.084600,-118.153090,75.000000,1.000000e+00,1.000000,0.230000,1.000000,1.000000
50%,2.689516e+07,1.375051e+08,40.672440,-77.033370,121.000000,2.000000e+00,8.000000,0.790000,1.000000,125.000000
75%,3.898107e+07,1.960659e+10,40.749210,-73.954640,201.000000,6.000000e+00,36.000000,2.050000,4.000000,299.000000
max,5.736742e+07,9.876313e+10,47.734620,-70.995950,24999.000000,1.000000e+08,1024.000000,90.000000,593.000000,3677.000000


In [12]:
missing_summary = pd.DataFrame({
    "null_count": df.isnull().sum(),
    "empty_string_count": (df == "").sum()
})

missing_summary["total_missing"] = (
    missing_summary["null_count"] + missing_summary["empty_string_count"]
)

missing_summary.sort_values("total_missing", ascending=False)

,null_count,empty_string_count,total_missing
last_review,328629,0,328629
neighbourhood_group,115874,0,115874
price,102599,0,102599
reviews_per_month,64481,0,64481
availability_365,880,0,880
host_name,439,0,439
minimum_nights,422,0,422
calculated_host_listings_count,319,0,319
name,278,0,278
number_of_reviews,183,0,183


In [18]:
df["id"].duplicated().sum()
df[df["id"].duplicated(keep=False)].head()


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
251,10085566,Cozy private furnished basement eff mins to town.,29820165,Kathleen,None,28805,35.59999,-82.51583,Entire home/apt,50.0,30.0,12.0,None,0.29,1.0,0.0
573,19050500,West Asheville Hilltop Hideout,28404139,Blake,None,28806,35.57100,-82.62420,Entire home/apt,45.0,1.0,186.0,None,4.64,1.0,0.0
641,20456656,EZ Access 2 the Heart of AVL & Blue Ridge Pkwy.,134950985,Bill & Linda,None,28803,35.55203,-82.52378,Entire home/apt,54.0,1.0,226.0,None,6.03,1.0,217.0
1374,34478448,Asheville Mountain Views King Beds Hot Tub!,3665372,Asheville Abode,None,28806,35.63172,-82.65745,Entire home/apt,270.0,3.0,70.0,None,4.43,42.0,331.0
2906,2989616,!South Austin Private room2,7567467,Chalu,None,78739,30.18864,-97.88932,Private room,35.0,30.0,34.0,None,0.55,9.0,298.0


In [ ]:
df["price"].value_counts().head(10)

price
100.0    5826
150.0    5722
75.0     4710
50.0     4401
80.0     3908
125.0    3809
200.0    3800
99.0     3791
60.0     3757
65.0     3546
Name: count, dtype: int64

In [22]:
df["room_type"].value_counts().head(10)


room_type
Entire home/apt    207874
Private room       112443
Shared room          6255
Hotel room           2057
Name: count, dtype: int64

In [23]:
df["neighbourhood_group"].value_counts().head(10)


neighbourhood_group
Manhattan              64028
Brooklyn               60105
Queens                 19013
City of Los Angeles    17544
Other Cities           11463
Maui                    7897
Honolulu                6313
Hawaii                  4977
Bronx                   3897
Kauai                   3247
Name: count, dtype: int64

In [27]:
df["price"].describe()


count    226030.000000
mean        219.716529
std         570.353609
min           0.000000
25%          75.000000
50%         121.000000
75%         201.000000
max       24999.000000
Name: price, dtype: float64

In [32]:
df[
    (df["latitude"] < -90) | (df["latitude"] > 90) |
    (df["longitude"] < -180) | (df["longitude"] > 180)
]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365


In [33]:
df[df["minimum_nights"] <= 0]


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365


In [ ]:
df[df["availability_365"] > 365]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
226035,1004098,Large Cozy 1 BR Apartment In Midtown East,45498551794,Michelle,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,NaN,3.0,74.0,None,0.59,1.0,374.0
226039,1006307,Cozy Clean Guest Room - Family Apt,75527839483,Carl,Manhattan,Upper West Side,40.80178,-73.96723,Private room,NaN,2.0,118.0,None,0.99,1.0,375.0
226056,1015144,"MAISON DES SIRENES1,bohemian apartment",48387947293,Miranda,Brooklyn,Bedford-Stuyvesant,40.68371,-73.94028,Entire home/apt,NaN,2.0,88.0,None,0.73,2.0,372.0
226065,1019562,Loft in Williamsburg Area w/ Roof,50968902783,John,Brooklyn,Greenpoint,40.72219,-73.93762,Private room,NaN,4.0,197.0,None,1.65,3.0,383.0
226066,1020114,back room/bunk beds,25066620900,Alfred,Manhattan,Harlem,40.82130,-73.95318,Private room,NaN,3.0,273.0,None,2.37,3.0,411.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328612,6085810,Private room in the best part of BK,4746552394,Brad,Brooklyn,Crown Heights,40.66743,-73.94712,Private room,NaN,7.0,49.0,None,1.01,1.0,425.0
328614,6086914,2 block walk from the beach in NYC,70051719594,Vladimir,Queens,Arverne,40.59270,-73.78990,Entire home/apt,NaN,2.0,89.0,None,1.82,1.0,394.0
328618,6089124,"Loft Space for Events, Meetings & Shoots",85557432222,Kalin,Manhattan,Flatiron District,40.74068,-73.98999,Entire home/apt,NaN,1.0,177.0,None,3.78,1.0,410.0
328625,6092990,Best Location near Columbia U,77864383453,Mifan,Manhattan,Morningside Heights,40.80460,-73.96545,Private room,NaN,1.0,1.0,None,0.02,2.0,395.0


In [37]:
df[
    df["last_review"] > pd.Timestamp.today()
]


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365


In [36]:
df["reviews_per_month"].quantile([0.01, 0.99])


0.01    0.02
0.99    7.34
Name: reviews_per_month, dtype: float64

In [ ]:
df["room_type"].value_counts(dropna=False)

room_type
Entire home/apt    207874
Private room       112443
Shared room          6255
Hotel room           2057
Name: count, dtype: int64